<a href="https://colab.research.google.com/github/ArjunRAj77/ICD10-code-extractor/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.5 MB/s 
     |████████████████████████████████| 451 kB 10.0 MB/s 
     |████████████████████████████████| 212 kB 60.8 MB/s 
     |████████████████████████████████| 182 kB 68.5 MB/s 
     |████████████████████████████████| 132 kB 65.7 MB/s 
     |████████████████████████████████| 127 kB 68.3 MB/s 
     |████████████████████████████████| 7.6 MB 55.8 MB/s 
     |████████████████████████████████| 1.3 MB 55.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import list_datasets
list_datasets()

# ICD 10 Extractor



In [64]:
import pandas as pd

icdcodedf = pd.read_csv('/content/ICD10.csv')
features=["ICDCode","Disease"]
icd10 = icdcodedf[features]
diseaselist = icd10['Disease'].tolist()
icdlist = icd10['ICDCode'].tolist()

icdcodedf

,ICDCode,Disease,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,A00.0,Cholera due to Vibrio cholerae 01,NaN,NaN,NaN
1,A00.1,Cholera due to Vibrio cholerae 01,NaN,NaN,NaN
2,A00.9,Cholera,NaN,NaN,NaN
3,A01.00,Typhoid fever,NaN,NaN,NaN
4,A01.1,Paratyphoid fever A,NaN,NaN,NaN
...,...,...,...,...,...
14810,C90.11,Plasma cell leukemia in remission,NaN,NaN,NaN
14811,C90.12,Plasma cell leukemia in relapse,NaN,NaN,NaN
14812,C88.8,Other malignant immunoproliferative diseases,NaN,NaN,NaN
14813,C90.21,Extramedullary plasmacytoma in remission,NaN,NaN,NaN


Cleaning the above dataset into a labelled dataset with string values.

In [90]:
icdfinallist=[]
diseasefinallist=[]
datalen=len(icd10)
for i in range(7000):
  str_X = str(icdlist[i])
  str_Y = str(diseaselist[i])
  icdfinallist.append(str_X)
  diseasefinallist.append(str_Y)

new_df=zip(icdfinallist,diseasefinallist)
column_names = ['ICDCode', 'Disease']
df = pd.DataFrame(new_df,columns = column_names)

df

,ICDCode,Disease
0,A00.0,Cholera due to Vibrio cholerae 01
1,A00.1,Cholera due to Vibrio cholerae 01
2,A00.9,Cholera
3,A01.00,Typhoid fever
4,A01.1,Paratyphoid fever A
...,...,...
6995,L02.529,Furuncle unspecified hand
6996,L02.33,Carbuncle of buttock
6997,L02.429,Furuncle of limb
6998,L02.629,Furuncle of unspecified foot


Converting the above dataframe into a new CSV file.

In [91]:
df.to_csv('/content/new_ICD10.csv')

*make sure that the entire dataset values are in string format, otherwise the following codes will not work.*

Loading a custom dataset in python:


Reference: https://huggingface.co/docs/datasets/v2.7.1/en/package_reference/loading_methods#datasets.load_dataset

In [92]:
from datasets import load_dataset
ICDData = load_dataset('csv', data_files='/content/new_ICD10.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a41ee7cbf0c6996a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [93]:
ICDData

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 7000
    })
})

In [84]:
ICDData['train'][0]

{'Unnamed: 0': 0,
 'ICDCode': 'A00.0',
 'Disease': 'Cholera due to Vibrio cholerae 01'}

In [94]:
ICDData['train'] = ICDData['train'].shuffle(seed=1).select(range(6000))
ICDData['train']

Dataset({
    features: ['Unnamed: 0', 'ICDCode', 'Disease'],
    num_rows: 6000
})

In [96]:
ICDData_train_validation = ICDData['train'].train_test_split(train_size=0.8)
ICDData_train_validation

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 1200
    })
})

In [97]:
ICDData_train_validation['test']

Dataset({
    features: ['Unnamed: 0', 'ICDCode', 'Disease'],
    num_rows: 1200
})

In [14]:
ICDData_train_validation['validation'] = ICDData_train_validation.pop('test')
ICDData_train_validation

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 100
    })
})

In [101]:
ICDData.update(ICDData_train_validation)
ICDData

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['Unnamed: 0', 'ICDCode', 'Disease'],
        num_rows: 1200
    })
})

In [102]:
ICDData['test'] = ICDData['test'].shuffle(seed=1).select(range(400))
ICDData['test']

Dataset({
    features: ['Unnamed: 0', 'ICDCode', 'Disease'],
    num_rows: 400
})

In [ ]:
pip install transformers

In [105]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-cased"
# checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(batch):
    return tokenizer(batch["Disease"], padding=True, truncation=True)

ICDData_encoded = ICDData.map(tokenize_function, batched=True, batch_size=None)

ICDData_encoded

OSError: ignored

In [ ]:
pip install torchvision

In [57]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 2
model = (AutoModelForSequenceClassification
         .from_pretrained(checkpoint, num_labels=num_labels)
         .to(device))

OSError: ignored

In [61]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(ICDData_encoded["train"]) // batch_size
model_name = f"{checkpoint}-icdCode-encoder"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )
training_args

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_n

In [62]:
import transformers
import re

[x for x in dir(transformers) if re.search(r'^AutoModel', x)]

['AutoModel',
 'AutoModelForAudioClassification',
 'AutoModelForAudioFrameClassification',
 'AutoModelForAudioXVector',
 'AutoModelForCTC',
 'AutoModelForCausalLM',
 'AutoModelForDepthEstimation',
 'AutoModelForDocumentQuestionAnswering',
 'AutoModelForImageClassification',
 'AutoModelForImageSegmentation',
 'AutoModelForInstanceSegmentation',
 'AutoModelForMaskedImageModeling',
 'AutoModelForMaskedLM',
 'AutoModelForMultipleChoice',
 'AutoModelForNextSentencePrediction',
 'AutoModelForObjectDetection',
 'AutoModelForPreTraining',
 'AutoModelForQuestionAnswering',
 'AutoModelForSemanticSegmentation',
 'AutoModelForSeq2SeqLM',
 'AutoModelForSequenceClassification',
 'AutoModelForSpeechSeq2Seq',
 'AutoModelForTableQuestionAnswering',
 'AutoModelForTokenClassification',
 'AutoModelForVideoClassification',
 'AutoModelForVision2Seq',
 'AutoModelForVisualQuestionAnswering',
 'AutoModelForZeroShotObjectDetection',
 'AutoModelWithLMHead']

In [ ]:
ICDData_encoded

In [ ]:
ICDData_encoded["train"]

In [ ]:
ICDData_encoded["validation"]

In [59]:
from transformers import Trainer

torch.cuda.empty_cache()

trainer = Trainer(model=model, 
                  args=training_args, 
                  train_dataset=ICDData_encoded["train"],
                  eval_dataset=ICDData_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train()

ValueError: ignored

In [ ]:
preds = trainer.predict(ICDData_encoded['validation'])
preds

In [ ]:
preds.predictions.shape

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(preds.label_ids, preds.predictions.argmax(axis=-1))

In [ ]:
def get_accuracy(preds):
  predictions = preds.predictions.argmax(axis=-1)
  labels = preds.label_ids
  accuracy = accuracy_score(preds.label_ids, preds.predictions.argmax(axis=-1))
  return {'accuracy': accuracy}

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

In [63]:
model_name

'distilbert-base-cased-icdCode-encoder'

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model_name)
classifier('Hepatitis A with hepatic coma')

In [ ]:
classifier('Hepatitis A without hepatic coma')